Codigo para ejecutar la ruta desde Visual Studio Code

In [1]:
import os
import sys

ruta =os.getcwd()
sys.path.append(ruta)

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Importar librerias necesarias
import sqlite3 as sql
import pandas as pd
import a_funciones as fn ### importar modulo de  funciones
from a_funciones import * ### importar toddas llas funciones del archivo

Carga de archivos desde carpeta bases de datos RRHH

In [4]:
employees = ("bases de datos RRHH/employee_survey_data.csv")
general = ("bases de datos RRHH/general_data.csv")
manager = ("bases de datos RRHH/manager_survey.csv")
retirement = ("bases de datos RRHH/retirement_info.csv")

Asignar los archivos cargados al df correspondiente

In [5]:
df_encuesta_empleado = pd.read_csv(employees)
df_general = pd.read_csv(general)
df_encuesta_gerente = pd.read_csv(manager)
df_retiro = pd.read_csv(retirement)

## df_encuesta_empleado

Analisis del df encuesta empleado

In [6]:
df_encuesta_empleado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8820 entries, 0 to 8819
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   EmployeeID               8820 non-null   int64  
 1   EnvironmentSatisfaction  8770 non-null   float64
 2   JobSatisfaction          8780 non-null   float64
 3   WorkLifeBalance          8744 non-null   float64
 4   DateSurvey               8820 non-null   object 
dtypes: float64(3), int64(1), object(1)
memory usage: 344.7+ KB


Descripción del df encuesta empleado, calculando la mediana para posible imputacion de datos nulos o faltantes.

In [19]:
# Filtrar las columnas numéricas
df_numerico = df_encuesta_empleado.select_dtypes(include=['float64'])

# Obtener el resumen estadístico con describe()
descripcion = df_numerico.describe()

# Calcular la mediana de cada columna numérica
mediana = df_numerico.median()

# Añadir la mediana al resumen estadístico
descripcion.loc['median'] = mediana

# Mostrar el resumen con la mediana incluida
print(descripcion)

        EnvironmentSatisfaction  JobSatisfaction  WorkLifeBalance
count               8770.000000      8780.000000      8744.000000
mean                   2.723603         2.728246         2.761436
std                    1.092694         1.101190         0.706205
min                    1.000000         1.000000         1.000000
25%                    2.000000         2.000000         2.000000
50%                    3.000000         3.000000         3.000000
75%                    4.000000         4.000000         3.000000
max                    4.000000         4.000000         4.000000
median                 3.000000         3.000000         3.000000


Imputacion de valores nulos (df_encuesta_empleado)   

In [20]:
# Filtrar las columnas numéricas y agregar la columna 'DateSurvey'
df_float = df_encuesta_empleado[['EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance', 'DateSurvey']]

# Calcular la mediana para cada columna por grupo 'DateSurvey'
medianas_por_fecha = df_float.groupby('DateSurvey').median()

# Función para imputar valores nulos con la mediana de su columna y grupo 'DateSurvey'
def imputar_valores_nulos(row):
    # Imputar valores nulos en 'EnvironmentSatisfaction'
    if pd.isna(row['EnvironmentSatisfaction']):
        row['EnvironmentSatisfaction'] = medianas_por_fecha.loc[row['DateSurvey'], 'EnvironmentSatisfaction']
    
    # Imputar valores nulos en 'JobSatisfaction'
    if pd.isna(row['JobSatisfaction']):
        row['JobSatisfaction'] = medianas_por_fecha.loc[row['DateSurvey'], 'JobSatisfaction']
    
    # Imputar valores nulos en 'WorkLifeBalance'
    if pd.isna(row['WorkLifeBalance']):
        row['WorkLifeBalance'] = medianas_por_fecha.loc[row['DateSurvey'], 'WorkLifeBalance']
    
    return row

# Aplicar la imputación de valores nulos
df_imputado = df_float.apply(imputar_valores_nulos, axis=1)

# Reemplazar las columnas en el DataFrame original con los valores imputados
df_encuesta_empleado[['EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance']] = df_imputado[['EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance']]

# Verificar el resultado
print(df_encuesta_empleado.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8820 entries, 0 to 8819
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   EmployeeID               8820 non-null   int64  
 1   EnvironmentSatisfaction  8820 non-null   float64
 2   JobSatisfaction          8820 non-null   float64
 3   WorkLifeBalance          8820 non-null   float64
 4   DateSurvey               8820 non-null   object 
dtypes: float64(3), int64(1), object(1)
memory usage: 344.7+ KB
None


## df_general

Analisis del df general

In [7]:
df_general.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8820 entries, 0 to 8819
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Age                      8820 non-null   int64  
 1   BusinessTravel           8820 non-null   object 
 2   Department               8820 non-null   object 
 3   DistanceFromHome         8820 non-null   int64  
 4   Education                8820 non-null   int64  
 5   EducationField           8820 non-null   object 
 6   EmployeeCount            8820 non-null   int64  
 7   EmployeeID               8820 non-null   int64  
 8   Gender                   8820 non-null   object 
 9   JobLevel                 8820 non-null   int64  
 10  JobRole                  8820 non-null   object 
 11  MaritalStatus            8820 non-null   object 
 12  MonthlyIncome            8820 non-null   int64  
 13  NumCompaniesWorked       8782 non-null   float64
 14  Over18                  

Columnas con valores nulos y conteo de registros

In [27]:
# Contar los valores nulos en cada columna del DataFrame
conteo_nulos = df_general.isnull().sum()

# Filtrar las columnas que tienen valores nulos
columnas_con_nulos = conteo_nulos[conteo_nulos > 0]

# Mostrar el resultado
print(columnas_con_nulos)

NumCompaniesWorked    38
TotalWorkingYears     18
dtype: int64


Descripción de las dos columnas con valores nulos para posible imputación con alguna de las métricas

In [30]:
# Filtrar las columnas numéricas
df_numerico = df_general[['NumCompaniesWorked', 'TotalWorkingYears']]

# Obtener el resumen estadístico con describe()
descripcion = df_numerico.describe()

# Calcular la mediana de cada columna específica
mediana = df_numerico.median()

# Añadir la mediana al resumen estadístico
descripcion.loc['median'] = mediana

# Mostrar el resumen con la mediana incluida
print(descripcion)

        NumCompaniesWorked  TotalWorkingYears
count          8782.000000        8802.000000
mean              2.694830          11.279936
std               2.498745           7.781780
min               0.000000           0.000000
25%               1.000000           6.000000
50%               2.000000          10.000000
75%               4.000000          15.000000
max               9.000000          40.000000
median            2.000000          10.000000


Despues de analisis se considera realizar la siguiente imputación:

Para NumCompaniesWorked (38 valores nulos): si YearsAtCompany es igual a TotalWorkingYears ponga 1, sino que ponga 2.
Para TotalWorkingYears (18 valores nulos): Que impute los nulos igual a YearsAtCompany (Por tener años trabajando en la empresa)

In [31]:
# Función para imputar valores nulos en NumCompaniesWorked
def imputar_num_companies_worked(row):
    if pd.isna(row['NumCompaniesWorked']):
        if row['YearsAtCompany'] == row['TotalWorkingYears']:
            return 1
        else:
            return 2
    return row['NumCompaniesWorked']

# Función para imputar valores nulos en TotalWorkingYears
def imputar_total_working_years(row):
    if pd.isna(row['TotalWorkingYears']):
        return row['YearsAtCompany']
    return row['TotalWorkingYears']

# Aplicar las funciones de imputación al DataFrame
df_general['NumCompaniesWorked'] = df_general.apply(imputar_num_companies_worked, axis=1)
df_general['TotalWorkingYears'] = df_general.apply(imputar_total_working_years, axis=1)

# Verificar el resultado
print(df_general[['NumCompaniesWorked', 'TotalWorkingYears']].isnull().sum())

NumCompaniesWorked    0
TotalWorkingYears     0
dtype: int64


Eliminación de columnas no necesarias en df_general:

EmployeeCount, Over18,  StandardHours = por que solo tienen un unico dato o valor

In [38]:
columnas_eliminar = ['EmployeeCount','Over18','StandardHours']

df_general = df_general.drop(columns=columnas_eliminar)

df_general.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8820 entries, 0 to 8819
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Age                      8820 non-null   int64  
 1   BusinessTravel           8820 non-null   object 
 2   Department               8820 non-null   object 
 3   DistanceFromHome         8820 non-null   int64  
 4   Education                8820 non-null   int64  
 5   EducationField           8820 non-null   object 
 6   EmployeeID               8820 non-null   int64  
 7   Gender                   8820 non-null   object 
 8   JobLevel                 8820 non-null   int64  
 9   JobRole                  8820 non-null   object 
 10  MaritalStatus            8820 non-null   object 
 11  MonthlyIncome            8820 non-null   int64  
 12  NumCompaniesWorked       8820 non-null   float64
 13  PercentSalaryHike        8820 non-null   int64  
 14  StockOptionLevel        

## df_encuesta_gerente

Analisis del df encuesta gerente

In [8]:
df_encuesta_gerente.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8820 entries, 0 to 8819
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   EmployeeID         8820 non-null   int64 
 1   JobInvolvement     8820 non-null   int64 
 2   PerformanceRating  8820 non-null   int64 
 3   SurveyDate         8820 non-null   object
dtypes: int64(3), object(1)
memory usage: 275.8+ KB


No tiene valores nulos el df encuesta gerente

## df_retiro

Analisis del df retiro

In [9]:
df_retiro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 711 entries, 0 to 710
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   EmployeeID         711 non-null    int64 
 1   Attrition          711 non-null    object
 2   retirementDate     711 non-null    object
 3   retirementType     711 non-null    object
 4   resignationReason  641 non-null    object
dtypes: int64(1), object(4)
memory usage: 27.9+ KB


Como las salidas por despido (fired) no tienen razon o motivo de renuncia se imputaran con la misma tipologia (fired).
La columna "Attrition" Se tomara como la variable objetivo, puesto que se refiere a la salida de empleados de una empresa, ya sea por renuncia, despido, jubilacion u otras razones.

In [37]:
df_retiro['resignationReason'] = df_retiro['resignationReason'].fillna('Fired')

print(df_retiro.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 711 entries, 0 to 710
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   EmployeeID         711 non-null    int64 
 1   Attrition          711 non-null    object
 2   retirementDate     711 non-null    object
 3   retirementType     711 non-null    object
 4   resignationReason  711 non-null    object
dtypes: int64(1), object(4)
memory usage: 27.9+ KB
None


## Creación y ejecución de Base de Datos
## Consultas SQL

In [43]:
con=sql.connect('bases de datos RRHH\\datos.db')  ### crea base de datos
cur=con.cursor()  ### para ejecutar consultas en la base de datos

In [44]:
####llevar datos a la base de datos ####

retiros=df_retiro
retiros.to_sql('retiros',con,index=False, if_exists='replace') ### para llevar un data frame de pandas a SQL
general=df_general
general.to_sql('general',con,index=False, if_exists='replace')
employee_survey=df_encuesta_empleado
employee_survey.to_sql('employee_survey',con,index=False, if_exists='replace')
manager_survey=df_encuesta_gerente
manager_survey.to_sql('manager_survey',con,index=False, if_exists='replace')


8820

In [45]:
cur.execute("Select name from sqlite_master where type='table'") ### consultar bases de datos
cur.fetchall()

[('retiros',), ('general',), ('employee_survey',), ('manager_survey',)]

In [52]:
cur.execute("""
    CREATE TABLE completo AS 
    SELECT a.*, b.*, c.*, d.* 
    FROM general a 
    LEFT JOIN employee_survey b 
        ON a.EmployeeID = b.EmployeeID AND a.InfoDate = b.DateSurvey 
    LEFT JOIN manager_survey c 
        ON a.EmployeeID = c.EmployeeID AND a.InfoDate = c.SurveyDate 
    LEFT JOIN retiros d 
        ON a.EmployeeID = d.EmployeeID
""")

In [53]:
cur.execute("Select name from sqlite_master where type='table'") ### consultar bases de datos
cur.fetchall()

[('retiros',),
 ('general',),
 ('employee_survey',),
 ('manager_survey',),
 ('completo',)]

In [54]:
pd.read_sql(""" select * from completo
""", con=con)

,Age,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeID,Gender,JobLevel,JobRole,...,DateSurvey,EmployeeID:2,JobInvolvement,PerformanceRating,SurveyDate,EmployeeID:3,Attrition,retirementDate,retirementType,resignationReason
0,51,Travel_Rarely,Sales,6,2,Life Sciences,1,Female,1,Healthcare Representative,...,2015-12-31,1,3,3,2015-12-31,NaN,None,None,None,None
1,31,Travel_Frequently,Research & Development,10,1,Life Sciences,2,Female,1,Research Scientist,...,2015-12-31,2,2,4,2015-12-31,2.0,Yes,2016-10-15,Resignation,Others
2,32,Travel_Frequently,Research & Development,17,4,Other,3,Male,4,Sales Executive,...,2015-12-31,3,3,3,2015-12-31,NaN,None,None,None,None
3,38,Non-Travel,Research & Development,2,5,Life Sciences,4,Male,3,Human Resources,...,2015-12-31,4,2,3,2015-12-31,NaN,None,None,None,None
4,32,Travel_Rarely,Research & Development,10,1,Medical,5,Male,1,Sales Executive,...,2015-12-31,5,3,3,2015-12-31,NaN,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8815,42,Travel_Rarely,Research & Development,5,4,Medical,4406,Female,1,Research Scientist,...,2016-12-31,4406,3,3,2016-12-31,NaN,None,None,None,None
8816,29,Travel_Rarely,Research & Development,2,4,Medical,4407,Male,1,Laboratory Technician,...,2016-12-31,4407,2,3,2016-12-31,NaN,None,None,None,None
8817,25,Travel_Rarely,Research & Development,25,2,Life Sciences,4408,Male,2,Sales Executive,...,2016-12-31,4408,3,4,2016-12-31,NaN,None,None,None,None
8818,42,Travel_Rarely,Sales,18,2,Medical,4409,Male,1,Laboratory Technician,...,2016-12-31,4409,2,3,2016-12-31,NaN,None,None,None,None


In [42]:
##con.close()